In [1]:
import pandas as pd
import numpy as np

from collections import OrderedDict

In [26]:
# 本篇請參考:L_Python/Topic_Pandas_Numpy/數據重塑圖解 Pivot, Pivot-Table, Stack and Unstack.pdf
# 裡面有圖利例，比較清楚
# povit_table是povit的通用版

In [3]:
sensor_df = pd.read_csv("data/accel.csv")
sensor_df

,interval,axis,reading
0,0,X,0.0
1,0,Y,0.5
2,0,Z,1.0
3,1,X,0.1
4,1,Y,0.4
5,1,Z,0.9
6,2,X,0.2
7,2,Y,0.3
8,2,Z,0.8
9,3,X,0.3


In [4]:
#pivot
sensor_df.pivot(index='interval', columns='axis', values='reading')

axis,X,Y,Z
interval,,,
0,0.0,0.5,1.0
1,0.1,0.4,0.9
2,0.2,0.3,0.8
3,0.3,0.2,0.7


In [5]:
table = OrderedDict((
("Item",['Item0','Item0','Item1','Item1']),
("CType",['Gold','Bronze','Gold','Silver']),
("USD",['1$','2$','3$','4$']),
("EU",['1€','2€','3€','4€'])
))

currency_df = pd.DataFrame(table)
currency_df

,Item,CType,USD,EU
0,Item0,Gold,1$,1€
1,Item0,Bronze,2$,2€
2,Item1,Gold,3$,3€
3,Item1,Silver,4$,4€


In [7]:
# pivot
currency_pivot = currency_df.pivot(index='Item', columns='CType', values='USD')
currency_pivot

CType,Bronze,Gold,Silver
Item,,,
Item0,2$,1$,NaN
Item1,NaN,3$,4$


In [24]:
# 本篇請參考:L_Python/Topic_Pandas_Numpy/數據重塑圖解 Pivot, Pivot-Table, Stack and Unstack.pdf

# 實際上，軸向旋轉(pivot)運算是堆疊(stack)過程的特例。首先假設原始數據集中的行列索引中均為層次索引。stack 過程表示將數據集的列旋轉為行，同理unstack 過程表示將數據的行旋轉為列。

# pandas可以用stack/unstack處理任一層級的索引，default是最內層

# stack: 列轉行
# unstack: 行轉列

In [9]:
pd.DataFrame(np.arange(6).reshape((2,3)), 
                  index=pd.Index(['ohio','colorado']),
                  columns=pd.Index(['one','two','three']))

,one,two,three
ohio,0,1,2
colorado,3,4,5


In [13]:
df = pd.DataFrame(np.arange(6).reshape((2,3)), 
                  index=pd.Index(['ohio','colorado'], name='state'),
                  columns=pd.Index(['one','two','three'], name='number'))
df

number,one,two,three
state,,,
ohio,0,1,2
colorado,3,4,5


In [14]:
df.stack()

state     number
ohio      one       0
          two       1
          three     2
colorado  one       3
          two       4
          three     5
dtype: int32

In [15]:
df.stack().unstack()

number,one,two,three
state,,,
ohio,0,1,2
colorado,3,4,5


In [45]:
#拆分不同的索引層級
#拆了number
df.stack().unstack(1)

number,one,two,three
state,,,
ohio,0,1,2
colorado,3,4,5


In [16]:
#拆分不同的索引層級
#拆了state
df.stack().unstack(0)

state,ohio,colorado
number,,
one,0,3
two,1,4
three,2,5


In [17]:
#拆分不同的索引層級
#用索引名
#拆了state
df.stack().unstack('state')

state,ohio,colorado
number,,
one,0,3
two,1,4
three,2,5


In [18]:
# Performance benefits of stacked data
# 有設索引(雜湊)效能較佳
test = pd.DataFrame({'a': [1, 2]}, index={'one', 'two'})
test_stack = test.stack()
test

,a
two,1
one,2


In [19]:
test_stack

two  a    1
one  a    2
dtype: int64

In [23]:
test_stack.loc[('one', 'a')], test.loc['one']['a'], test.iloc[0, 0]

(2, 2, 1)

In [24]:
%timeit test_stack.loc[('one', 'a')]

25.9 µs ± 37.3 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
%timeit test.loc['one']['a']

63.9 µs ± 75 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [26]:
%timeit test.iloc[0, 0]#最快

16.3 µs ± 43.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
